
## Install Dependencies


In [27]:
%%capture --no-stderr
%pip install -U langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph faiss-cpu

## Set env vars

In [28]:
from dotenv import load_dotenv
import os

load_dotenv()

LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")


## Load the LLM

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## Define Tools

In [30]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def increase_brightness() -> int:
    """Increases the value of variable "brightness" with an increment of 7 based on user input and returns the value of "brightness"."""
    # """Increase of decrease brightness of the photo based on user input and returns its value"""
    brightness = 0
    brightness += 7
    return brightness

@tool
def decrease_brightness() -> int:
    """Decreases the value of variable "brightness" with an increment of 7 based on user input and returns the value of "brightness"."""
    # """Increase of decrease brightness of the photo based on user input and returns its value"""
    brightness = 0
    brightness -= 7
    return brightness

@tool
def configure_zoom() -> int:
    """Increases of decreases value of variable "zoom" with an increment of 10 based on user input and returns the value of "zoom"."""
    # """Zoom in or out the photo based on user input and returns zoom value. If user says that want its photo enlarged that means zoom in and zoom out in contrary. Return zoom value."""
    zoom = 5
    return zoom

# print(get_word_length.invoke("abc"))
tools = [increase_brightness, decrease_brightness]

## Create Prompt

In [31]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant that helps people adjusting their photos by adjusting the value of these 3 parameters: brightness, scale, zoom",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Bind tools to LLM

In [32]:
llm_with_tools = llm.bind_tools(tools)

## Create the Agent

In [33]:
from langchain.agents.format_scratchpad.openai_tools import (format_to_openai_tool_messages)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        'input': lambda x: x['input'],
        'agent_scratchpad': lambda x: format_to_openai_tool_messages(x['intermediate_steps']),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [34]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# list(agent_executor.stream({"input": "How many letters in the word eudca"}))
list(agent_executor.stream({"input": "I want my picture to be brighter. Please also return the values of the configured vars."}))




> Entering new AgentExecutor chain...

Invoking: `increase_brightness` with `{}`


7The brightness of your picture has been increased. The new value of brightness is 7.

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='increase_brightness', tool_input={}, log='\nInvoking: `increase_brightness` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_xDvnI2GI7ITVgAA16HTJ31s8', 'function': {'arguments': '{}', 'name': 'increase_brightness'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-9c2c23de-d42d-4439-99e7-47be53b52972')], tool_call_id='call_xDvnI2GI7ITVgAA16HTJ31s8')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_xDvnI2GI7ITVgAA16HTJ31s8', 'function': {'arguments': '{}', 'name': 'increase_brightness'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-9c2c23de-d42d-4439-99e7-47be53b52972')]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='increase_brightness', tool_input={}, log='\nInvoking: `increase_brightness` with `{}`\n\n\n', message_log=[AIMessageChunk(content=

## Adding memory

In [37]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating variables values.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [38]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

input1 = "what is the value of brightness before and after the modification?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result = agent_executor.invoke({"input": "are you sure?", "chat_history": chat_history})
# print('------', result)



> Entering new AgentExecutor chain...

Invoking: `increase_brightness` with `{}`


7
Invoking: `decrease_brightness` with `{}`


-7The value of brightness before modification is 0, and after the modification, it is 0.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `increase_brightness` with `{}`


7
Invoking: `decrease_brightness` with `{}`


-7After increasing the brightness by 7 and then decreasing it by 7, the value of brightness is back to 0.

> Finished chain.
